<a href="https://colab.research.google.com/github/jhbeann/Bigdata_R/blob/main/Chapter3_section1_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 제 2유형: 데이터 분석

## SECTION 01. 빅데이터 분석 과정

### 1. 빅데이터 분석 과정 이해
  1) 필요 패키지 import 
  
  - dplyr : filter , select, arrange 
  - ggplot2 : scatter plot, histogram, boxplot
  - 분류
    - 의사결정나무 : library(rpart)
    - knn : library(class)
    - svm : install.packages("e1071") & library(nnet)
    - 로지스틱회귀 : library(nnet)
    - 랜덤포레스트 : install.packages("randomForest") & library(randomForest)
  - 회귀 : 선형회귀 lm()
  - 군집 : k-means
  - 연관 : install.packages("arules") & library(arules)
  

In [1]:
library(dplyr)
library(ggplot2)
# 의사결정나무 분류모델
library(rpart)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
df <- read.csv("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv")

# 목표 :: iris 를 이용하여 꽃잎의 길이나 넓이에 따라 붓꽃의 종류를 구별하는 것

In [3]:
# 데이터 살펴보기
head(df,5)

,sepal_length,sepal_width,petal_length,petal_width,species
,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa


In [4]:
str(df) # 구조정보

'data.frame':	150 obs. of  5 variables:
 $ sepal_length: num  5.1 4.9 4.7 4.6 5 5.4 4.6 5 4.4 4.9 ...
 $ sepal_width : num  3.5 3 3.2 3.1 3.6 3.9 3.4 3.4 2.9 3.1 ...
 $ petal_length: num  1.4 1.4 1.3 1.5 1.4 1.7 1.4 1.5 1.4 1.5 ...
 $ petal_width : num  0.2 0.2 0.2 0.2 0.2 0.4 0.3 0.2 0.2 0.1 ...
 $ species     : chr  "setosa" "setosa" "setosa" "setosa" ...


In [5]:
summary(df)  # 기술통계

  sepal_length    sepal_width     petal_length    petal_width   
 Min.   :4.300   Min.   :2.000   Min.   :1.000   Min.   :0.100  
 1st Qu.:5.100   1st Qu.:2.800   1st Qu.:1.600   1st Qu.:0.300  
 Median :5.800   Median :3.000   Median :4.350   Median :1.300  
 Mean   :5.843   Mean   :3.057   Mean   :3.758   Mean   :1.199  
 3rd Qu.:6.400   3rd Qu.:3.300   3rd Qu.:5.100   3rd Qu.:1.800  
 Max.   :7.900   Max.   :4.400   Max.   :6.900   Max.   :2.500  
   species         
 Length:150        
 Class :character  
 Mode  :character  
                   
                   
                   

In [6]:
# 데이터 전처리
# species 컬럼 : 레이블 인코딩
# 'setosa':1, 'versicolor':2, 'virginica':3
df$species <- as.numeric(factor(df$species), level=c("setosa","versicolor","virginica"))
df$species

[1] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 [38] 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 [75] 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3
[112] 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
[149] 3 3

#### 5) 데이터셋 준비
  - 학습용 데이터셋 : 테스트용 데이터셋 = 8 : 2
  - sample()
  

In [7]:
# 난수 발생 위해 seed 부여
set.seed(123)

#  전체 행의 80% 해당하는 인덱스 선정
idx <- sample(1:nrow(df),0.8*nrow(df))

# 인덱스 행 : 학습데이터
df_train <- df[idx, ]
# 나머지는 테스트 데이터
df_test <- df[-idx, ]

In [8]:
print(dim(df_train))
print(dim(df_test))

[1] 120   5
[1] 30  5


In [10]:
# rpart() 의사결정나무
# 종속변수~ 독립변수 + 독립변수...
# species~. : species종속변수 나머지는 독립변수

model <- rpart(species~., method = "class", data = df_train)

In [11]:
# 학습된 모델을 기반한 예측수행
pred <- predict(model, df_test, type="class")

In [15]:
# 예측결과와 실제값을 비교
cm <- table(df_test$species, pred)
print(cm)

# 2를 3으로 2개 잘못 분류함. 대각선 값들이 정확히 분류한 것

   pred
     1  2  3
  1 10  0  0
  2  0 13  2
  3  0  0  5


In [14]:
# 모델평가 - 정확도
# 1을 1로 2를 2로 3을 3으로 맞춘 합 => 정확도
acc <- (cm[1,1] + cm[2,2] + cm[3,3])/nrow(df_test)
print(acc)

# 분류(예측)모델의 정확도는 0.933..
# 93%의 정확도로 붓꽃의 품종을 분류할 수 있다.

[1] 0.9333333


#### 2. 분석모델 성능 평가 방법
  - 오차행렬  : caret 패키지의 confusionMatrix()
  

In [18]:
# 모델 평가 - 오차행렬, caret 패키지 사용
install.packages("caret")
library(caret)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘clock’, ‘recipes’


Loading required package: lattice



In [19]:
confusionMatrix(data=pred, reference = factor(df_test$species), mode = "prec_recall" )

Confusion Matrix and Statistics

          Reference
Prediction  1  2  3
         1 10  0  0
         2  0 13  0
         3  0  2  5

Overall Statistics
                                          
               Accuracy : 0.9333          
                 95% CI : (0.7793, 0.9918)
    No Information Rate : 0.5             
    P-Value [Acc > NIR] : 4.34e-07        
                                          
                  Kappa : 0.8947          
                                          
 Mcnemar's Test P-Value : NA              

Statistics by Class:

                     Class: 1 Class: 2 Class: 3
Precision              1.0000   1.0000   0.7143
Recall                 1.0000   0.8667   1.0000
F1                     1.0000   0.9286   0.8333
Prevalence             0.3333   0.5000   0.1667
Detection Rate         0.3333   0.4333   0.1667
Detection Prevalence   0.3333   0.4333   0.2333
Balanced Accuracy      1.0000   0.9333   0.9600